# API Binance


https://api.binance.com/api/v1/ticker/24hr

## Function connect_to_redshift

In [2]:
import psycopg2
import logging
import pandas as pd
from pandas.io.json import json_normalize
import requests, json
from datetime import datetime
from psycopg2.extras import execute_values

def connect_to_redshift(host, dbname, user, password, port=5439):
    """
    Establishes a connection to a Redshift database.

    Parameters:
    - host (str): The hostname or IP address of the Redshift cluster.
    - dbname (str): The name of the Redshift database to connect to.
    - user (str): The username for authenticating with the Redshift database.
    - password (str): The password for authenticating with the Redshift database.
    - port (int, optional): The port number on which the Redshift cluster is listening. Default is 5439.

    Returns:
    - conn: A psycopg2 connection object representing the connection to the Redshift database.

    Raises:
    - Exception: If an error occurs during the connection attempt.
    """
    try:
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        logging.info("Connection established")
        return conn
    except Exception as error:
        logging.info(f"An error occurred: {error}")


2024-04-01 18:07:27,713 - INFO - Libraries calls ok


## Function execute_query

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Libraries calls ok")

def execute_query(connection, query):
    """
    Executes a SQL query on the provided database connection.

    Parameters:
    - connection: psycopg2 connection object representing the connection to the database.
    - query (str): The SQL query to be executed.

    Returns:
    - None

    Raises:
    - Exception: If an error occurs during the query execution.
    """
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        logging.info("Query executed successfully")
    except Exception as error:
        logging.info(f"An error occurred: {error}")


2024-04-01 18:07:27,731 - INFO - Libraries calls ok


### Truncate table mateobelossi_coderhouse.binance_coins if exists.

In [4]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Libraries calls ok")

dbname = "data-engineer-database"
port = 5439
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
table_name = "binance_coins"
schema_name = "mateobelossi_coderhouse"
user = "mateobelossi_coderhouse"
with open("/home/mateo/Desktop/Curso DE/✍️Consignas de Pre.entregas y Proyecto Final-20240331T125607Z-001/1er_pre_entrega/password_redshift.txt ",'r') as f:
    password= f.read()

logging.info("Connecting to redshift...")
conn = connect_to_redshift(host, dbname, user, password, port)
cur = conn.cursor()

logging.info("Checking if table exists...")
cur.execute(f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_schema = '{schema_name}' AND table_name = '{table_name}')")
table_exists = cur.fetchone()[0]
logging.info(f"Table exists ? : {table_exists}")

if table_exists:
    logging.info(f"Truncating table {schema_name}.{table_name}")
    query = f"truncate table {schema_name}.{table_name}"
    execute_query(conn, query)
else:
    logging.info(f"table {schema_name}.{table_name} do not exist.")

cur.close()
conn.close()

2024-04-01 18:07:27,836 - INFO - Libraries calls ok
2024-04-01 18:07:27,838 - INFO - Connecting to redshift...


2024-04-01 18:07:29,267 - INFO - Connection established
2024-04-01 18:07:29,270 - INFO - Checking if table exists...
2024-04-01 18:07:29,677 - INFO - Table exists ? : True
2024-04-01 18:07:29,682 - INFO - Truncating table mateobelossi_coderhouse.binance_coins
2024-04-01 18:07:30,126 - INFO - Query executed successfully


### Create table mateobelossi_coderhouse.binance_coins if not exits.

In [5]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Libraries calls ok")

dbname = "data-engineer-database"
user = "mateobelossi_coderhouse"
with open("/home/mateo/Desktop/Curso DE/✍️Consignas de Pre.entregas y Proyecto Final-20240331T125607Z-001/1er_pre_entrega/password_redshift.txt ",'r') as f:
    password= f.read()
port = 5439
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
table_name = "binance_coins"
schema_name = "mateobelossi_coderhouse"

CREATE_TABLE =f"""
CREATE TABLE {user}.{table_name} (
    symbol VARCHAR(256),
    priceChange FLOAT,
    priceChangePercent FLOAT,
    weightedAvgPrice FLOAT,
    prevClosePrice FLOAT,
    lastPrice FLOAT,
    lastQty FLOAT,
    bidPrice FLOAT,
    bidQty FLOAT,
    askPrice FLOAT,
    askQty FLOAT,
    openPrice FLOAT,
    highPrice FLOAT,
    lowPrice FLOAT,
    volume FLOAT,
    quoteVolume FLOAT,
    openTime BIGINT,
    closeTime BIGINT,
    firstId BIGINT,
    lastId BIGINT,
    count BIGINT,
    created_at TIMESTAMP
);
"""

logging.info("Connecting to redshift...")
conn = connect_to_redshift(host, dbname, user, password, port)
cur = conn.cursor()

logging.info("Checking if table exists...")
cur.execute(f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_schema = '{schema_name}' AND table_name = '{table_name}')")
table_exists = cur.fetchone()[0]
logging.info(f"Table exists ? : {table_exists}")

if not table_exists:
    logging.info(f"Creating table {schema_name}.{table_name}")
    cur.execute(
    CREATE_TABLE
    )
    conn.commit()
else:
    logging.info(f"table {schema_name}.{table_name} already exists.")

cur.close()
conn.close()

2024-04-01 18:07:30,187 - INFO - Libraries calls ok
2024-04-01 18:07:30,194 - INFO - Connecting to redshift...


2024-04-01 18:07:31,442 - INFO - Connection established
2024-04-01 18:07:31,445 - INFO - Checking if table exists...
2024-04-01 18:07:31,850 - INFO - Table exists ? : True
2024-04-01 18:07:31,852 - INFO - table mateobelossi_coderhouse.binance_coins already exists.


#### 1) requests to api binance https://api.binance.com/api/v1/ticker/24hr
#### 2) Include a column named "created_at" in the obtained results, indicating the current time at the moment of making the requests.
#### 3) Save the results in a CSV file.
#### 4) Insert the results into Redshift.

In [6]:
dbname = "data-engineer-database"
user = "mateobelossi_coderhouse"
with open("/home/mateo/Desktop/Curso DE/✍️Consignas de Pre.entregas y Proyecto Final-20240331T125607Z-001/1er_pre_entrega/password_redshift.txt ",'r') as f:
    password= f.read()
port = 5439
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
table_name = "binance_coins"
schema_name = "mateobelossi_coderhouse"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Libraries calls ok")

CUR_TIME = datetime.now().strftime("%Y%m%d_%H%M%S")
logging.info(f"Start Time : {CUR_TIME} ")


logging.info(f"Making the requests to API binance.")
r = requests.get('https://api.binance.com/api/v1/ticker/24hr')

if r.status_code == 200:
    logging.info(f"Connection ok ; Requests Status: {r.status_code}")
    result = r.json()
    result = json_normalize(result)

    logging.info("Adding a column named created_at with current time.")
    result['created_at'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    logging.info("Loading data from binance to a csv")
    name_csv = 'mercado_binance.csv'
    file_path = str("./binance/") + str(CUR_TIME) + '_' + str(name_csv)
    pd.DataFrame(result).to_csv(file_path, index=False)
    
    logging.info("Loading data from binance to redshift")
    
    conn = connect_to_redshift(host, dbname, user, password, port)
    data_to_insert = [tuple(row) for row in result.values]
    insert_query = f"""
        INSERT INTO {schema_name}.{table_name} (
            symbol, priceChange, priceChangePercent, weightedAvgPrice, prevClosePrice, lastPrice, lastQty,
            bidPrice, bidQty, askPrice, askQty, openPrice, highPrice, lowPrice, volume, quoteVolume,
            openTime, closeTime, firstId, lastId, count, created_at
        ) VALUES %s;
        """
    with conn.cursor() as cur:
        execute_values(
            cur,
            insert_query,
            data_to_insert,
            page_size=len(data_to_insert)
        )
        conn.commit()
        logging.info("Data inserted into Redshift successfully.")

else:
    logging.info(f"It was not able to connect to Binance ; Requests Status: {r.status_code}")

2024-04-01 18:07:36,312 - INFO - Libraries calls ok
2024-04-01 18:07:36,318 - INFO - Start Time : 20240401_180736 
2024-04-01 18:07:36,321 - INFO - Making the requests to API binance.
2024-04-01 18:07:37,530 - INFO - Connection ok ; Requests Status: 200
/tmp/ipykernel_4468/580455300.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  result = json_normalize(result)
2024-04-01 18:07:37,638 - INFO - Adding a column named created_at with current time.
2024-04-01 18:07:37,670 - INFO - Loading data from binance to a csv
2024-04-01 18:07:37,778 - INFO - Loading data from binance to redshift
2024-04-01 18:07:39,011 - INFO - Connection established
2024-04-01 18:07:41,641 - INFO - Data inserted into Redshift successfully.


## Reading the data from Redshift.

In [7]:
dbname = "data-engineer-database"
user = "mateobelossi_coderhouse"
with open("/home/mateo/Desktop/Curso DE/✍️Consignas de Pre.entregas y Proyecto Final-20240331T125607Z-001/1er_pre_entrega/password_redshift.txt ",'r') as f:
    password= f.read()
port = 5439
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
table_name = "binance_coins"
schema_name = "mateobelossi_coderhouse"

conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

select_query = f"""
SELECT * FROM {schema_name}.{table_name}
"""

df = pd.read_sql(select_query, conn)

conn.close()


df.head()


/home/mateo/.local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,symbol,pricechange,pricechangepercent,weightedavgprice,prevcloseprice,lastprice,lastqty,bidprice,bidqty,askprice,...,highprice,lowprice,volume,quotevolume,opentime,closetime,firstid,lastid,count,created_at
0,ETHBTC,-0.001200,-2.339,0.050660,0.051300,0.050100,0.4904,0.050100,21.7642,0.050110,...,0.051340,0.049850,39090.1671,1980.290029,1711919256711,1712005656711,438971731,439081873,110143,2024-04-01 18:07:37
1,LTCBTC,-0.000066,-4.438,0.001512,0.001489,0.001421,4.1670,0.001420,0.7900,0.001421,...,0.001590,0.001417,204989.0440,309.841946,1711919256665,1712005656665,96353740,96411965,58226,2024-04-01 18:07:37
2,BNBBTC,-0.000247,-2.883,0.008415,0.008564,0.008319,0.1130,0.008319,3.8110,0.008320,...,0.008600,0.008286,49785.0580,418.922755,1711919256685,1712005656685,238145920,238243276,97357,2024-04-01 18:07:37
3,NEOBTC,-0.000004,-1.801,0.000231,0.000228,0.000224,13.4900,0.000223,14.5900,0.000224,...,0.000237,0.000221,73820.8800,17.088082,1711919243445,1712005643445,46258075,46263252,5178,2024-04-01 18:07:37
4,QTUMETH,-0.000029,-2.200,0.001334,0.001320,0.001289,1.5000,0.001286,114.2000,0.001289,...,0.001355,0.001270,9772.0000,13.035371,1711919256674,1712005656674,5438851,5439029,179,2024-04-01 18:07:37


## ESTO DE ABAJO LO AGREGUÉ YO (MATEO) PARA VER SI SE PUEDEN OCULTAR LOS DATOS CON ESTO. IGUAL VOS PA SI DECIS QUE CON EL ARCHIVO TXT YA SE OCULTAN LISTO JOYA. Y SINO IGUAL, SI SE COMPLICA MUCHO, LA PASSWORD NO LA PONGO Y SE LA ESCRIBO EN EL "DETALLE" DE LA ENTREGA (DIJO QUE PODÍAMOS HACER ESO EL PROFE)

In [2]:
%ls .env

.env


In [6]:
%pip install dotenv

Defaulting to user installation because normal site-packages is not writeable
  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [64 lines of output]
      /usr/lib/python3/dist-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [16 lines of output]
            Traceback (most recent call last):
              File "<string>", line 2, in <module>
              File "<pip-setuptools-caller>", line 14, in <module>
              File "/tmp/pip-wheel-cgzyrswg/distribute_53b372fb43a443ad9fc3b09d6e19ce76/setuptools/__init__.py", line 2, in <module>
                fro

In [4]:
import os
from dotenv import load_dotenv
load_dotenv() # Cargar las variables de entorno
url = os.environ.get("DB_HOST")
data_base = os.environ.get("DB_NAME")
user = os.environ.get("DB_USER")
password = os.environ.get("DB_PASSWORD")
db_port = os.environ.get("DB_PORT")
print(url, '\n',user,'\n',password,'\n', db_port)

ModuleNotFoundError: No module named 'dotenv'